In [69]:
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm import tqdm
import json
import os

In [89]:
sector_tickers = 'B C Y P E F V I K U R'.split()
sector_tickers[-1] += 'E'
sector_tickers_list = ['XL' + x for x in sector_tickers]
market_ticker = 'SPX'
sector_tickers = ' '.join(sector_tickers_list)
sectors = yf.Tickers(sector_tickers)
market = yf.Ticker(market_ticker)

In [90]:
hist = sectors.history(period='10y')
market_hist = market.history(period='11y')
hist['Close'][sector_tickers_list]

[*********************100%***********************]  11 of 11 completed


,XLB,XLC,XLY,XLP,XLE,XLF,XLV,XLI,XLK,XLU,XLRE
Date,,,,,,,,,,,
2012-05-25,27.672440,NaN,38.014492,25.951824,45.627071,9.432727,30.857342,28.726879,23.938347,25.341536,NaN
2012-05-29,28.152760,NaN,38.541485,26.135607,46.390671,9.567286,30.959019,29.113527,24.256157,25.419447,NaN
2012-05-30,27.648005,NaN,37.935444,25.921183,44.968548,9.345260,30.781096,28.628164,24.101540,25.263636,NaN
2012-05-31,27.534031,NaN,37.856384,25.867588,44.576241,9.425997,30.628593,28.521221,23.989887,25.398203,NaN
2012-06-01,26.947844,NaN,36.661842,25.392809,43.462349,9.076138,30.086338,27.731480,23.362862,25.291964,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2022-05-18,82.040001,57.950001,142.800003,71.830002,81.720001,33.200001,127.940002,91.540001,132.220001,71.690002,41.939999
2022-05-19,82.620003,57.869999,142.899994,70.550003,81.419998,32.980000,128.220001,90.720001,130.770004,71.540001,41.860001
2022-05-20,82.419998,58.020000,140.410004,70.699997,81.769997,32.919998,129.779999,89.739998,131.059998,71.739998,42.349998


In [28]:
hist.columns
price_features = ['Open', 'High', 'Low', 'Close', 'Volume']
## Features to engineer: 
## NOTE: Everything below is calculated based on daily returns, not prices.
##     SMA: 5, 13, 21, 55, 243
##     Volatility: std, ADR
##     Financials: quareterly_financials
##     Temporal Features: DOW, M, Q
##     Overall Market Features: SMA: 21, 144, 300 
##                              Vol: std: 13, 55 
##                              PC Ratio
##                              Exchange Rates (EURO/USD, FRANC/USD, AUD/USD)
##                                             SMA: 8, 21
##                                             Vol: std - 8, 21, 89

In [110]:
new = yf.Ticker('SPY')

('2022-05-25',
 '2022-05-27',
 '2022-05-31',
 '2022-06-01',
 '2022-06-03',
 '2022-06-06',
 '2022-06-08',
 '2022-06-10',
 '2022-06-13',
 '2022-06-15',
 '2022-06-17',
 '2022-06-21',
 '2022-06-22',
 '2022-06-24',
 '2022-06-27',
 '2022-06-30',
 '2022-07-01',
 '2022-07-15',
 '2022-08-19',
 '2022-09-16',
 '2022-09-30',
 '2022-11-18',
 '2022-12-16',
 '2022-12-30',
 '2023-01-20',
 '2023-03-17',
 '2023-03-31',
 '2023-06-16',
 '2023-12-15',
 '2024-01-19',
 '2024-12-20')

# Feature Engineering

In [71]:
def get_static_features(df, tickers):
    for ticker in tickers:
        df['Returns', ticker] = 1 + (df['Close', ticker] - 
                                     df['Open', ticker]) / df['Open', ticker]
    return df

def get_rolling_features(df, tickers, periods=[5, 13, 21, 55, 243]):
    for ticker in tqdm(tickers):
        for per in periods:
            df[f'{per}_SMA', ticker] = df['Returns', ticker].rolling(window=per).mean()
            df[f'{per}_vol', ticker] = df['Returns', ticker].rolling(window=per).std()
    return df

def get_market_data(df):
    df['Returns'] = 1 + (df['Close'] - df['Open']) / df['Open']
    for per in [21, 144, 300]:
        df[f'{per}_SMA'] = df['Returns'].rolling(window=per).mean()
    for per in [13, 55]:
        df[f'{per}_vol'] = df['Returns'].rolling(window=per).std()
    pcr = pd.read_csv('data/pcr.csv')
    return df
    

df = get_returns(hist, sector_tickers_list)
get_moving_averages(df, sector_tickers_list)

100%|██████████████████████████████████████████| 12/12 [00:00<00:00, 313.82it/s]


Close                                                         \
                SPX        XLB        XLC        XLE        XLF        XLI   
Date                                                                         
2012-05-24  0.07222  27.786409        NaN  45.844231   9.466367  28.924314   
2012-05-25  0.08254  27.672428        NaN  45.627060   9.432725  28.726883   
2012-05-29  0.08254  28.152773        NaN  46.390663   9.567285  29.113537   
2012-05-30  0.08254  27.647995        NaN  44.968552   9.345261  28.628160   
2012-05-31  0.07222  27.534033        NaN  44.576229   9.425993  28.521217   
...             ...        ...        ...        ...        ...        ...   
2022-05-18      NaN  82.040001  57.950001  81.720001  33.200001  91.540001   
2022-05-19      NaN  82.620003  57.869999  81.419998  32.980000  90.720001   
2022-05-20      NaN  82.419998  58.020000  81.769997  32.919998  89.739998   
2022-05-23      NaN  83.930000  58.889999  83.930000  33.990002  91.209999   
2022-05-24      NaN  82.300003  55.889999  83.035004  33.345001  89.480003   

                                                         ...     5_SMA  \
                   XLK        XLP       XLRE        XLU  ...      XLRE   
Date                                                     ...             
2012-05-24   23.989883  25.974791        NaN  25.256548  ...       NaN   
2012-05-25   23.938351  25.951820        NaN  25.341539  ...       NaN   
2012-05-29   24.256149  26.135605        NaN  25.419447  ...       NaN   
2012-05-30   24.101545  25.921185        NaN  25.263628  ...       NaN   
2012-05-31   23.989883  25.867580        NaN  25.398199  ...       NaN   
...                ...        ...        ...        ...  ...       ...   
2022-05-18  132.220001  71.830002  41.939999  71.690002  ...  0.997538   
2022-05-19  130.770004  70.550003  41.860001  71.540001  ...  0.996964   
2022-05-20  131.059998  70.699997  42.349998  71.739998  ...  0.994424   
2022-05-23  134.070007  72.160004  42.900002  72.599998  ...  0.996479   
2022-05-24  130.179993  72.227501  42.340000  72.470001  ...  0.995350   

              13_SMA    21_SMA    55_SMA   243_SMA     5_SMA 13_SMA 21_SMA  \
                XLRE      XLRE      XLRE      XLRE       SPX    SPX    SPX   
Date                                                                         
2012-05-24       NaN       NaN       NaN       NaN       NaN    NaN    NaN   
2012-05-25       NaN       NaN       NaN       NaN       NaN    NaN    NaN   
2012-05-29       NaN       NaN       NaN       NaN       NaN    NaN    NaN   
2012-05-30       NaN       NaN       NaN       NaN       NaN    NaN    NaN   
2012-05-31       NaN       NaN       NaN       NaN  0.976241    NaN    NaN   
...              ...       ...       ...       ...       ...    ...    ...   
2022-05-18  0.990787  0.991112  0.997664  0.999280       NaN    NaN    NaN   
2022-05-19  0.993380  0.990588  0.997498  0.999293       NaN    NaN    NaN   
2022-05-20  0.992913  0.991129  0.997421  0.999307       NaN    NaN    NaN   
2022-05-23  0.992389  0.992162  0.997221  0.999295       NaN    NaN    NaN   
2022-05-24  0.993413  0.992002  0.997394  0.999254       NaN    NaN    NaN   

           55_SMA 243_SMA  
              SPX     SPX  
Date                       
2012-05-24    NaN     NaN  
2012-05-25    NaN     NaN  
2012-05-29    NaN     NaN  
2012-05-30    NaN     NaN  
2012-05-31    NaN     NaN  
...           ...     ...  
2022-05-18    NaN     NaN  
2022-05-19    NaN     NaN  
2022-05-20    NaN     NaN  
2022-05-23    NaN     NaN  
2022-05-24    NaN     NaN  

[2517 rows x 156 columns]

In [93]:
pcr = pd.read_csv('data/pcr.csv', header=None)
pcr

,0,1
0,1162339200000,0.640000
1,1162425600000,0.610000
2,1162512000000,0.640000
3,1162771200000,0.550000
4,1162857600000,0.570000
...,...,...
3990,1652918400000,0.830000
3991,1653004800000,0.800000
3992,1653264000000,0.800000
3993,1653350400000,0.590000
